# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff5cfd32b80>
├── 'a' --> tensor([[-0.8143, -0.3311,  0.0344],
│                   [-1.7267,  0.7368,  1.0944]])
└── 'x' --> <FastTreeValue 0x7ff5cfd32040>
    └── 'c' --> tensor([[ 0.6703, -0.4713, -0.9282,  0.8941],
                        [ 0.1216,  0.1996,  0.8852,  1.5863],
                        [-0.8379,  1.2474, -1.4648, -0.8612]])

In [4]:
t.a

tensor([[-0.8143, -0.3311,  0.0344],
        [-1.7267,  0.7368,  1.0944]])

In [5]:
%timeit t.a

76.8 ns ± 2.64 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff5cfd32b80>
├── 'a' --> tensor([[ 0.1326, -0.6931,  0.4406],
│                   [ 0.0732, -0.3790, -1.1068]])
└── 'x' --> <FastTreeValue 0x7ff5cfd32040>
    └── 'c' --> tensor([[ 0.6703, -0.4713, -0.9282,  0.8941],
                        [ 0.1216,  0.1996,  0.8852,  1.5863],
                        [-0.8379,  1.2474, -1.4648, -0.8612]])

In [7]:
%timeit t.a = new_value

90.8 ns ± 2.74 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8143, -0.3311,  0.0344],
               [-1.7267,  0.7368,  1.0944]]),
    x: Batch(
           c: tensor([[ 0.6703, -0.4713, -0.9282,  0.8941],
                      [ 0.1216,  0.1996,  0.8852,  1.5863],
                      [-0.8379,  1.2474, -1.4648, -0.8612]]),
       ),
)

In [10]:
b.a

tensor([[-0.8143, -0.3311,  0.0344],
        [-1.7267,  0.7368,  1.0944]])

In [11]:
%timeit b.a

79.7 ns ± 1.59 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0500, -0.4617, -0.5972],
               [ 1.6755,  1.5010, -0.8057]]),
    x: Batch(
           c: tensor([[ 0.6703, -0.4713, -0.9282,  0.8941],
                      [ 0.1216,  0.1996,  0.8852,  1.5863],
                      [-0.8379,  1.2474, -1.4648, -0.8612]]),
       ),
)

In [13]:
%timeit b.a = new_value

668 ns ± 7.37 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.13 µs ± 23.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.4 µs ± 110 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

283 µs ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

269 µs ± 6.16 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff5cfd32c40>
├── 'a' --> tensor([[[-0.8143, -0.3311,  0.0344],
│                    [-1.7267,  0.7368,  1.0944]],
│           
│                   [[-0.8143, -0.3311,  0.0344],
│                    [-1.7267,  0.7368,  1.0944]],
│           
│                   [[-0.8143, -0.3311,  0.0344],
│                    [-1.7267,  0.7368,  1.0944]],
│           
│                   [[-0.8143, -0.3311,  0.0344],
│                    [-1.7267,  0.7368,  1.0944]],
│           
│                   [[-0.8143, -0.3311,  0.0344],
│                    [-1.7267,  0.7368,  1.0944]],
│           
│                   [[-0.8143, -0.3311,  0.0344],
│                    [-1.7267,  0.7368,  1.0944]],
│           
│                   [[-0.8143, -0.3311,  0.0344],
│                    [-1.7267,  0.7368,  1.0944]],
│           
│                   [[-0.8143, -0.3311,  0.0344],
│                    [-1.7267,  0.7368,  1.0944]]])
└── 'x' --> <FastTreeValue 0x7ff5cfd32100>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

50 µs ± 1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff5cccaa3d0>
├── 'a' --> tensor([[-0.8143, -0.3311,  0.0344],
│                   [-1.7267,  0.7368,  1.0944],
│                   [-0.8143, -0.3311,  0.0344],
│                   [-1.7267,  0.7368,  1.0944],
│                   [-0.8143, -0.3311,  0.0344],
│                   [-1.7267,  0.7368,  1.0944],
│                   [-0.8143, -0.3311,  0.0344],
│                   [-1.7267,  0.7368,  1.0944],
│                   [-0.8143, -0.3311,  0.0344],
│                   [-1.7267,  0.7368,  1.0944],
│                   [-0.8143, -0.3311,  0.0344],
│                   [-1.7267,  0.7368,  1.0944],
│                   [-0.8143, -0.3311,  0.0344],
│                   [-1.7267,  0.7368,  1.0944],
│                   [-0.8143, -0.3311,  0.0344],
│                   [-1.7267,  0.7368,  1.0944]])
└── 'x' --> <FastTreeValue 0x7ff5cccaa3a0>
    └── 'c' --> tensor([[ 0.6703, -0.4713, -0.9282,  0.8941],
                        [ 0.1216,  0.1996,  0.8852,  1.5863],
                 

In [23]:
%timeit t_cat(trees)

45.3 µs ± 1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

84 µs ± 894 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.8143, -0.3311,  0.0344],
                [-1.7267,  0.7368,  1.0944]],
       
               [[-0.8143, -0.3311,  0.0344],
                [-1.7267,  0.7368,  1.0944]],
       
               [[-0.8143, -0.3311,  0.0344],
                [-1.7267,  0.7368,  1.0944]],
       
               [[-0.8143, -0.3311,  0.0344],
                [-1.7267,  0.7368,  1.0944]],
       
               [[-0.8143, -0.3311,  0.0344],
                [-1.7267,  0.7368,  1.0944]],
       
               [[-0.8143, -0.3311,  0.0344],
                [-1.7267,  0.7368,  1.0944]],
       
               [[-0.8143, -0.3311,  0.0344],
                [-1.7267,  0.7368,  1.0944]],
       
               [[-0.8143, -0.3311,  0.0344],
                [-1.7267,  0.7368,  1.0944]]]),
    x: Batch(
           c: tensor([[[ 0.6703, -0.4713, -0.9282,  0.8941],
                       [ 0.1216,  0.1996,  0.8852,  1.5863],
                       [-0.8379,  1.2474, -1.4648, -0.8612]],
         

In [26]:
%timeit Batch.stack(batches)

105 µs ± 1.85 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.8143, -0.3311,  0.0344],
               [-1.7267,  0.7368,  1.0944],
               [-0.8143, -0.3311,  0.0344],
               [-1.7267,  0.7368,  1.0944],
               [-0.8143, -0.3311,  0.0344],
               [-1.7267,  0.7368,  1.0944],
               [-0.8143, -0.3311,  0.0344],
               [-1.7267,  0.7368,  1.0944],
               [-0.8143, -0.3311,  0.0344],
               [-1.7267,  0.7368,  1.0944],
               [-0.8143, -0.3311,  0.0344],
               [-1.7267,  0.7368,  1.0944],
               [-0.8143, -0.3311,  0.0344],
               [-1.7267,  0.7368,  1.0944],
               [-0.8143, -0.3311,  0.0344],
               [-1.7267,  0.7368,  1.0944]]),
    x: Batch(
           c: tensor([[ 0.6703, -0.4713, -0.9282,  0.8941],
                      [ 0.1216,  0.1996,  0.8852,  1.5863],
                      [-0.8379,  1.2474, -1.4648, -0.8612],
                      [ 0.6703, -0.4713, -0.9282,  0.8941],
                      [ 0.1216,  

In [28]:
%timeit Batch.cat(batches)

192 µs ± 2.21 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

552 µs ± 8.89 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
